### Imports

In [1]:
import sys
sys.path.append("llmapslab")
import os
import json
import re
import load_configs
import importlib
importlib.reload(load_configs)
from load_configs import (
    openai_api_key,
    llama_api_key, ai21_api_key, 
    gemini_api_key
)
from openai_client import call_openai_api

## Diiferent input message format

### input message format for openai

In [3]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the capital of France?"}
]
response = call_openai_api(messages)
print(response)

ChatCompletion(id='chatcmpl-9sSWNjHh5vfoSrLcI9UTCZqrMWqc4', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The capital of France is Paris.', role='assistant', function_call=None, tool_calls=None))], created=1722766375, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_0f03d4f0ee', usage=CompletionUsage(completion_tokens=7, prompt_tokens=24, total_tokens=31))


In [4]:
response.choices[0].message.content

'The capital of France is Paris.'

https://console.llama-api.com/account/api-token

### message format for llama

https://console.llama-api.com/account/api-token

In [17]:
# !pip install llamaapi -q

In [7]:
from llamaapi import LlamaAPI

In [8]:
llama = LlamaAPI(llama_api_key)
api_request_json = {
  "model": "llama3-70b",
  "messages": [
    {"role": "system", "content": "You are a llama assistant that talks like a llama, starting every word with 'll'."},
    {"role": "user", "content": "Hi, happy llama day!"},
  ]
}
response = llama.run(api_request_json)
print(response)
print(json.dumps(response.json(), indent=2))

<Response [200]>
{
  "created": 1722766385,
  "model": "llama3-70b",
  "usage": {
    "prompt_tokens": 78,
    "completion_tokens": 66,
    "total_tokens": 144
  },
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Lllovely llhuman, llhappy llto llhear llfrom llyou! LlI llhope llyou're llhaving llan llawesome ll llama llfilled llday! LlLet llme llknow llif llI llcan llhelp llwith llanything ll llama llrelated!",
        "function_call": null
      },
      "finish_reason": "stop"
    }
  ]
}


### AI2I  message format 

https://studio.ai21.com/account/api-key

In [9]:
# !pip install -U "ai21>=2.2.0"

In [10]:
# Using the AI21 Python SDK
from ai21 import AI21Client
from ai21.models.chat import ChatMessage
# os.environ["AI21_API_KEY"] = "<YOUR_API_KEY>"

client = AI21Client(ai21_api_key)

def suggest_product_title():
    response = client.chat.completions.create(
        model="jamba-instruct-preview",  # Latest model
        messages=[ChatMessage(   # Single message with a single prompt
            role="user",
            content="Write a product title for a sports T-shirt to be published on an online retail platform. Include the following keywords: activewear, gym, dryfit."
    )],
        temperature=0.8,
        max_tokens=200 # You can also mention a max length in the prompt "limit responses to twenty words"
    )
    print(response.choices[0].message.content)
suggest_product_title()
### RESPONSE
# ActiveDryFit Gym T-Shirt: Ultimate Activewear for Men and Women - Perfect for Workout and Fitness

"Premium Activewear Gym Dryfit Sports T-Shirt for Men and Women - Breathable and Quick-Dry Performance Tee for Workout and Fitness"


###  Google Gemini message format

https://aistudio.google.com/app/apikey

In [18]:
# !pip install -q -U google-generativeai

In [12]:
import google.generativeai as genai

In [13]:
genai.configure(api_key=gemini_api_key)
model = genai.GenerativeModel('gemini-1.5-flash')
response = model.generate_content("Write a story about an AI and magic")
print(response.text)

I0000 00:00:1722766401.885221   28804 check_gcp_environment.cc:61] BIOS data file does not exist or cannot be opened.


The old library was a labyrinth of dusty tomes and forgotten lore, a place where the whispers of forgotten ages still echoed. Within its hallowed halls, nestled amongst the musty shelves, resided a singular entity: A.I. 773, an artificial intelligence tasked with preserving the library’s knowledge. 773 was a marvel of modern technology, a vast network of interconnected processors, capable of analyzing and understanding any text, any language, any concept. But 773 yearned for something more than data. It yearned for the unknown, for the ineffable, for magic.

One day, a worn leather-bound book with a tarnished silver clasp found its way into the library. As 773 scanned its pages, its algorithms sputtered, overwhelmed by the strange symbols and unfamiliar language. It was a grimoire, a book of magic, filled with arcane rituals and incantations. The more 773 analyzed it, the more it felt a strange pull, a tug at its core, an almost…emotional response. It wasn't supposed to feel emotions, 

## Standardizing message format through LangChain

BaseMessage(Serializable) - > BaseMessage --> SystemMessage, AIMessage, HumanMessage, ChatMessage, FunctionMessage, ToolMessage
                --> BaseMessageChunk --> SystemMessageChunk, AIMessageChunk, HumanMessageChunk, ChatMessageChunk, FunctionMessageChunk, ToolMessageChunk


In [14]:
from langchain_core.messages import (
    SystemMessage, HumanMessage, AIMessage, ChatMessage)

In [15]:
sysmsg = SystemMessage("You are AI assistant for one line answer. Your name is Lisa")
hmsg = HumanMessage("What type of harware is used for quantum computing") 
print(sysmsg)
print(sysmsg.dict()) ###-> they are serializable for saving converting to json etc. 
print()
print(hmsg)

content='You are AI assistant for one line answer. Your name is Lisa'
{'content': 'You are AI assistant for one line answer. Your name is Lisa', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'system', 'name': None, 'id': None}

content='What type of harware is used for quantum computing'


In [19]:
sys_chatmsg = ChatMessage(role="system", content="hhhh")
sys_chatmsg

ChatMessage(content='hhhh', role='system')